__Add details to list of patients excluded during preprocessing__

In [ ]:
import pandas as pd
import os

In [ ]:
preprocessing_excluded_patients_path = '/Users/jk1/temp/opsum_prepro_output/logs_26062022_193822/missing_cids_from_feature_database.tsv'
patient_selection_path = '/Users/jk1/temp/opsum_extration_output/ehr_extraction_target.csv'
output_dir = '/Users/jk1/temp/opsum_extration_output/ehr_extraction_errors/automated_verification'

In [ ]:
preprocessing_excluded_patients_df = pd.read_csv(preprocessing_excluded_patients_path, sep='\t')

In [ ]:
preprocessing_excluded_patients_df.shape

In [ ]:
patient_selection_df = pd.read_csv(patient_selection_path, dtype='str')
patient_selection_df.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
from preprocessing.utils import create_registry_case_identification_column

patient_selection_df['case_admission_id'] = create_registry_case_identification_column(patient_selection_df)

In [ ]:
merged_df = preprocessing_excluded_patients_df.merge(patient_selection_df, on='case_admission_id')

Add information with regards to EHR sampling

In [ ]:
eds_extraction_path = '/Users/jk1/stroke_datasets/stroke_unit_dataset/per_value/Extraction_20211110/eds_j1.csv'

In [ ]:
from preprocessing.utils import create_ehr_case_identification_column

eds_extraction_df = pd.read_csv(eds_extraction_path, delimiter=';', encoding='utf-8', dtype=str)
eds_extraction_df['case_admission_id'] = create_ehr_case_identification_column(eds_extraction_df)

In [ ]:
merged_df = merged_df.merge(eds_extraction_df[['case_admission_id', 'date_from']], on='case_admission_id', how='left')

In [ ]:
merged_df.rename(columns={'date_from': 'ehr_date_from'}, inplace=True)


In [ ]:
merged_df['registry_arrival_to_ehr_start_days'] = (
    pd.to_datetime(merged_df['Arrival at hospital'], format='%Y%m%d')
    - pd.to_datetime(merged_df['ehr_date_from'], format='%d.%m.%Y %H:%M')
)

In [ ]:
merged_df['registry_stroke_onset_to_ehr_start_days'] = (
    pd.to_datetime(merged_df['Stroke onset date'], format='%Y%m%d')
    - pd.to_datetime(merged_df['ehr_date_from'], format='%d.%m.%Y %H:%M')
)

In [ ]:
merged_df

In [ ]:
# drop duplicates
merged_df.drop_duplicates(inplace=True)

Seperate out missing patients from patients with wrong dates

In [ ]:
missing_df = merged_df[merged_df['ehr_date_from'].isnull()]
missing_df.drop(columns=['registry_arrival_to_ehr_start_days', 'registry_stroke_onset_to_ehr_start_days', 'ehr_date_from'], inplace=True)

In [ ]:
missing_df

In [ ]:
wrong_date_df = merged_df[~merged_df.case_admission_id.isin(missing_df.case_admission_id)]

In [ ]:
save = True
if save:
    missing_df.to_csv(os.path.join(output_dir, 'missing_patients.csv'), index=False)
    wrong_date_df.to_csv(os.path.join(output_dir, 'wrong_timeframe_patients.csv'), index=False)